In [3]:
# 引用需要的library
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from matplotlib import pyplot as plt
import helper

In [4]:
# 从文本文件中读数据
# 读取到的内容
# X=评论（string）， y=情感分类（0/1）
helper.Read_comments_from_file()


In [ ]:
# 调用jieba分词
# 文本向量化：1. 类one-hot编码 2. count思想 3. tf-idf思想
# 返回 X=评论向量（list），y=情感分类（0/1）

In [6]:
# 切分数据集为X_train, y_train, X_test, y_test


Hello World


In [ ]:
# 数据预处理：标准化/归一化

In [ ]:
# 调用各分类模型，训练
# KNeighborsClassifier
# DecisionTreeClassifier
# SVC
# RandomForestClassifier
# AdaBoostClassifier
# GradientBoostingClassifier
# XGBClassifier
# LGBMClassifier

In [ ]:
# 调用训练完毕的模型，做预测

In [ ]:
# 计算各模型的acc，训练时间，预测时间

In [ ]:
# 保存模型到models文件夹